 # Random Search (RS)

In [1]:
from functions_SFMA import *

## Set parameters and load the initial datasets and W matrices

In [2]:
num_N = 6 # number of rows of a matrix 
nrep_it = 0.01 # number of iterations of RS
n_samp = 1 # number of sampling of RS
num_W_a = 2 # index of the first W matrix
num_W_b = 3 # index of the last W matrix

In [3]:
n_bit = num_N*num_K # number of spins (qubits)
n_in = n_bit # sizes of the initial datasets 
k_FM = int(n_bit/2)-1 # hyperparameter for FM models 

val_dim = f'd{n_bit}'
val_standard = "standard"
val_k = str(k_FM)
val_anneal = 'SA'

# Load the W matrices, the inital datasets, the mean and std values, and the objective values.

list_Matrix_W = [] # list for collecting the W Matrices
list_xs_int = [] # list for collecting the inital inputs 
list_ys_int = [] # list for collecting the inital outputs 
list_mean = [] # list for collecting the mean values
list_std = [] # list for collecting the std values
list_minBB_1st = [] # list for collecting the 1st optimal objective (output) values
list_minBB_2nd = [] # list for collecting the 2nd optimal objective values
list_minBB_mid2 = [] # list for collecting the mid2 optimal values

folder_data =  f'final_data_file_{n_bit}'
for n_W in range(d_W): # load the inital dataset, mean and std, and objective values for each W matrix  
    val_num_W = f'W_{n_W}'
    init_file = f'{folder_data}/initial_Data_{val_num_W}_{val_dim}_{val_standard}_{val_k}_{val_anneal}.bz2' # file containing the inital dataset
    with open(init_file, 'rb') as inD:
        initD = pickle.load(inD)
    
    Matrix_W = initD['Matrix_W'] # W matrix
    list_Matrix_W.append(Matrix_W) # append the W matrix to list_Matrix_W  
    xs_int = initD['xs_int'] # inital input
    list_xs_int.append(xs_int) # append xs_int to list_xs_int
    ys_int = initD['ys_int'] # inital output
    list_ys_int.append(ys_int) # append ys_int to list_ys_int  
    mean = initD['mean'] # mean
    list_mean.append(mean) # append mean to list_mean 
    std = initD['std'] # std
    list_std.append(std) # append std to list_std  
    minBB_1st = initD['minBB_1st'] # 1st optimal objective value
    list_minBB_1st.append(minBB_1st) # append minBB_1st to list_minBB_1st
    minBB_2nd = initD['minBB_2nd'] # 2nd optimal objective value
    list_minBB_2nd.append(minBB_2nd) # append minBB_2nd to list_minBB_2nd 
    minBB_mid2 = initD['minBB_mid2'] # mid2 optimal value
    list_minBB_mid2.append(minBB_mid2) # append minBB_mid2 to list_minBB_mid2 

## Perform RS and save the results

In [4]:
n_it = pow(n_bit,2) # unit of the number of iterations of RS
n_it_tot = int(n_it*nrep_it) # total number of the iterations
n_tot = n_in + 1 + n_it_tot # sizes of the final datasets
val_n_tot = str(n_tot)
# Run RS and augment the dataset.
for n_W in range(num_W_a, num_W_b): # RS for each W matrix
    #### Setup
    val_number_W = f'W_{n_W}'
    M_W = list_Matrix_W[n_W] # W matrix
    mean_W = list_mean[n_W] # mean
    std_W = list_std[n_W] # std
    minBB_1st_W = list_minBB_1st[n_W] # 1st optimal objective
    minBB_2nd_W = list_minBB_2nd[n_W] # 2nd optimal objective
    minBB_mid2_W = list_minBB_mid2[n_W] # mid2 optimal objective
    opt_1st = np.array([minBB_1st_W]*n_tot) # array of minBB_1st_W
    opt_2nd = np.array([minBB_2nd_W]*n_tot) # array of minBB_2nd_W
    opt_mid2 = np.array([minBB_mid2_W]*n_tot) # array of minBB_mid2_W 
    opt_1st_rounded = np.round(opt_1st, n_round) # rounded opt_1st 
    opt_2nd_rounded = np.round(opt_2nd, n_round) # rounded opt_2nd 
    opt_mid2_rounded = np.round(opt_mid2, n_round) # rounded opt_mid2 
    #
    xs_int_W, ys_int_W = list_xs_int[n_W][:-1], list_ys_int[n_W][:-1] # original inital dataset
    list_samp_xs =[] # list for samples of inputs (xs)
    list_samp_ys =[] # list for samples of outputs (ys)
    ################################################################ RS
    for samp in range(n_samp):
        xs, ys = Aug_RS_with_xin(M_W, num_N, num_K, xs_int_W, n_tot, samp) # dataset augmentation 
        list_samp_xs.append(xs) # append the final xs to list_samp_xs
        list_samp_ys.append(ys) # append the final ys to list_samp_ys 
    ################################################################ Statistical calculation
    ensmeble_xs = np.array(list_samp_xs) # n_samp samples of the final xs
    ensmeble_ys = np.array(list_samp_ys) # n_samp samples of the final ys
    ensmeble_ys_min = output_sample_min(ensmeble_ys) # n_samp samples of the final minimum ys (ys_min)
    ys_mean = np.mean(ensmeble_ys_min, axis=0) # averaged ys_min along the vertical direction
    ys_mean_rounded = np.round(ys_mean,n_round) # rounded ys_mean
    rate = success_rate(ensmeble_ys, n_in, n_tot, minBB_1st_W, n_samp) # success rate  
    rate_rounded = success_rate_rounded(ensmeble_ys, n_in, n_tot, n_round, minBB_1st_W, n_samp) # rounded success rate
    ################################################################ Save the results.
    dictionary = {'ensmeble_xs': ensmeble_xs, 'ensmeble_ys': ensmeble_ys,'ensmeble_ys_min': ensmeble_ys_min,
        'ys_mean': ys_mean, 'ys_mean_rounded': ys_mean_rounded, 'rate': rate, 'rate_rounded': rate_rounded,
        'opt_1st': opt_1st, 'opt_2nd': opt_2nd, 'opt_mid2': opt_mid2, 'opt_1st_rounded': opt_1st_rounded, 'opt_2nd_rounded': opt_2nd_rounded, 'opt_mid2_rounded': opt_mid2_rounded }
    
    file = f'{folder_data}/RS_result_{val_number_W}_{val_dim}_{val_n_tot}.bz2'

    with open(file, 'wb') as fl:
        pickle.dump(dictionary, fl)